# Basic Functions

In [1]:
import random
import time

# Function to generate random polynomial
# n: degree
# maxval: max value of polynomial coefficient
def GenerateGPolynomial(n, maxval=16):
    return [random.randint(0, maxval) for i in range(n)]

# Check the correctness of different algorithm output
# stdr: standard result
# r: test result
def check(stdr,r):
    if len(stdr) != len(r):
        raise ValueError("LENGTH ERROR")
    if sum([abs(stdr[i]-r[i]) for i in range(len(stdr))]):
        raise ValueError("VALUE ERROR")
    else:
        print("check pass!")

# Base Case solution: Schoolbook Multiplication

In [2]:
def SBMul(A,B):
    
    # Check the input degree (They should be the same)
    if len(A) != len(B):
        raise ValueError("Schoolbook Multiplictaion: Degree of A != Degree of B")
    
    # Get the degree information
    n = len(A)
    
    # Define an empty list to store the result
    C = [0 for i in range(2*n)] 
    
    # Compute the result
    for i in range(n):
        for j in range(n):
            C[i+j] = C[i+j]+A[i]*B[j]
    return C

# Algorithm 1: Karatsuba Method

In [3]:
def KMul(A,B):
    
    # Check the input degree (They should be the same)
    if len(A) != len(B):
        raise ValueError("Karatsuba: Degree of A != Degree of B")
    
    # Get the degree information
    n = len(A)
    
    # Define an empty list to store the result
    C = [0 for i in range(2*n)] 
    
    # Define the intermediate variables
    A0 = A[:n//2]
    A1 = A[n//2:]
    B0 = B[:n//2]
    B1 = B[n//2:]
    
    # Step 1
    Asum = [A0[i]+A1[i] for i in range(n//2)] 
    Bsum = [B0[i]+B1[i] for i in range(n//2)]
    
    # Step 2
    if n>8:
        M0 = KMul(A0,B0)
        M1 = KMul(A1,B1)
        MS = KMul(Asum,Bsum)
    else: #base case condition satisfied
        M0 = SBMul(A0,B0)
        M1 = SBMul(A1,B1)
        MS = SBMul(Asum,Bsum)
        
    # Step 3
    # (A0 + A1)(B0 + B1) - A0B0 - A1B1.
    P1 = [MS[i]-M1[i]-M0[i] for i in range(n)]
    
    # Step 4
    # A0B0.
    C[:n//2] = M0[:n//2]
    # A0B0+
    C[n//2:n] = [M0[n//2:n][i] + P1[:n//2][i] for i in range(n//2)]
    # A1B1+
    C[n:3*n//2] = [P1[n//2:n][i] + M1[:n//2][i] for i in range(n//2)]
    # A1B1.
    C[3*n//2:] = M1[n//2:]
    
    return C

## Test for KMul

In [4]:
A = GenerateGPolynomial(1024)
B = GenerateGPolynomial(1024)
check(SBMul(A,B),KMul(A,B))

check pass!


# Algorithm 2: Refined Karatsuba Method

In [5]:
def RKMul(A,B):
    
    # Check the input degree (They should be the same)
    if len(A) != len(B):
        raise ValueError("Refined Karatsuba: Degree of A != Degree of B")
    
    # Get the degree information
    n = len(A)
    
    # Define an empty list to store the result
    C = [0 for i in range(2*n)] 
    
    # Define the intermediate variables
    A0 = A[:n//2]
    A1 = A[n//2:]
    B0 = B[:n//2]
    B1 = B[n//2:]
    
    # Step 1
    #Put your codes here
    # A0 + A1.
    Asum = [A0[i]+A1[i] for i in range(n//2)] 
    # B0 + B1.
    Bsum = [B0[i]+B1[i] for i in range(n//2)]
    # print('Step1 Asum: ', Asum)
    # print('Step1 Bsum: ', Bsum)

    # Step 2
    if n>8:
        # A0B0.
        M0 = RKMul(A0,B0)
        # A1B1.
        M1 = RKMul(A1,B1)
        # (A0+A1)(B0+B1).
        MS = RKMul(Asum,Bsum)
    else:
        # A0B0.
        M0 = SBMul(A0,B0)
        # A1B1.
        M1 = SBMul(A1,B1)
        # (A0+A1)(B0+B1).
        MS = SBMul(Asum,Bsum)
        
    # Step 3
    # initialization.
    P1 = [0 for i in range(3*n//2)]
    #Put your codes here
    P1[:n//2] = M0[:n//2]
    P1[n//2:2*n//2] = [M0[n//2:n][i] - M1[:n//2][i] for i in range(n//2)]
    P1[2*n//2:] = [-M1[n//2:][i] for i in range(n//2)]
    #print('Step3 P1: ', P1)

    # Step 4
    P2 = [0 for i in range(2*n-2)]
    #Put your codes here
    P2[:n//2] = P1[:n//2]
    P2[n//2:3*n//2] = [P1[n//2:][i] - P1[:n][i] for i in range(n)]
    P2[3*n//2:] = [-P1[n:][i] for i in range(n//2)]
    #print('Step4 P2: ', P2)
    
    # Step 5
    #Put your codes here
    C[:n//2] = P2[:n//2]
    C[n//2:3*n//2] = [P2[n//2:3*n//2][i] + MS[i] for i in range(n)]
    C[3*n//2:] = P2[3*n//2:]
    #print('Step5 C: ', C)

    return C

## Test for RKMul

In [6]:
A = GenerateGPolynomial(1024)
B = GenerateGPolynomial(1024)
# A = [3,5,9,23,4,6,8,13]
# B = [34,67,87,4,67,3,8,45]

check(SBMul(A,B),RKMul(A,B))

check pass!


# Algorithm 3: Overlap-Free Karatsuba Method

In [7]:
def OFKMul(A,B):

    # Check the input degree (They should be the same)
    if len(A) != len(B):
        raise ValueError("Overlap-Free Karatsuba: Degree of A != Degree of B")
    
    # Get the degree information
    n = len(A)
    
    # Define an empty list to store the result
    C = [0 for i in range(2*n)] 
    
    # Define the intermediate variables
    #Put your codes here
    Aeven = []; Aodd = []; Beven = []; Bodd = []
    Aeven = [A[i] for i in range(n) if i%2==0]
    Aodd = [A[i] for i in range(n) if i%2==1]
    Beven = [B[i] for i in range(n) if i%2==0]
    Bodd = [B[i] for i in range(n) if i%2==1]
    # Aeven = []; Aodd = []; Beven = []; Bodd = []
    # for i in range(n):
    #     if i % 2 == 0:
    #         Aeven.append(A[i])
    #         Beven.append(B[i])
    #     else:
    #         Aeven.append(0)
    #         Beven.append(0)
    
    # Step 1
    #Put your codes here
    # Aeven + Aodd.
    Asum = [Aeven[i]+Aodd[i] for i in range(n//2)] 
    # Beven + Bodd.
    Bsum = [Beven[i]+Bodd[i] for i in range(n//2)]
    # print('Step1 Asum: ', Asum)
    # print('Step1 Bsum: ', Bsum)
    
    # Step 2
    if n>8:
        Meven = OFKMul(Aeven,Beven)
        Modd = OFKMul(Aodd,Bodd)
        MS = OFKMul(Asum,Bsum)
    else:
        Meven = SBMul(Aeven,Beven)
        Modd = SBMul(Aodd,Bodd)
        MS = SBMul(Asum,Bsum)
    
    # Step 3
    #Put your codes here
    Podd = [MS[i] - Meven[i] - Modd[i] for i in range(n)]
    #print('Step3 Podd: ', Podd)

    # Step 4
    Peven = [0 for i in range(n)]
    #Put your codes here
    Peven[:1] = [Meven[:1][i] for i in range(1)]
    Peven[1:n-1] = [Meven[1:][i] + Modd[:n-1][i] for i in range(n-1)]
    Peven[n-1:] = [Modd[n-2] for i in range(1)]
    #print('Step4 Peven: ', Peven)
    
    # Step 5
    #Put your codes here
    for i in range(0, n*2, 2):
        C[i] = Peven[i//2]
        C[i+1] = Podd[i//2]
    #print('Step5 C: ', C)
            
    return C

# Test for OFKMul

In [8]:
A = GenerateGPolynomial(1024)
B = GenerateGPolynomial(1024)
# A = [3,5,9,23,4,6,8,13,34,54,23,5,2,45,67,8]
# B = [34,67,87,4,67,3,8,45,23,4,5,78,9,23,4,56]

check(SBMul(A,B),OFKMul(A,B))

check pass!


# Algorithm 4: 2-Level 7-Way Karatsuba Method (Extension Project)

In [ ]:
def TLKMul(A,B):
    
    # Check the input degree (They should be the same)
    if len(A) != len(B):
        raise ValueError("2-Level 7-Way Karatsuba: Degree of A != Degree of B")
    
    # Get the degree information
    n = len(A)
    # Define 1/4 problem size
    m = n//4
    
    # Define an empty list to store the result
    C = [0 for i in range(2*n)] 
    
    # Define the intermediate variables
    A0 = A[0:m]
    A1 = A[m:2*m]
    A2 = A[2*m:3*m]
    A3 = A[3*m:n]
    B0 = B[0:m]
    B1 = B[m:2*m]
    B2 = B[2*m:3*m]
    B3 = B[3*m:n]
    
    # Step 1
    #Put your codes here
    #Alow =
    #Ahigh =
    #Aodd =
    #Aeven =
    #Blow =
    #Bhigh =
    #Bodd =
    #Beven =
    
    # Step 2
    #Put your codes here
    #AEO = 
    #BEO =
    
    # Step 3 & Step 4
    if n > 32:
        MEO = TLKMul(AEO,BEO)
        
        M0 = TLKMul(A0,B0)
        M1 = TLKMul(A1,B1)
        M2 = TLKMul(A2,B2)
        M3 = TLKMul(A3,B3)
        ML = TLKMul(Alow,Blow)
        MH = TLKMul(Ahigh,Bhigh)
        
    if n == 32:
        MEO = TLKMul(AEO,BEO)
        
        M0 = RKMul(A0,B0)
        M1 = RKMul(A1,B1)
        M2 = RKMul(A2,B2)
        M3 = RKMul(A3,B3)
        ML = RKMul(Alow,Blow)
        MH = RKMul(Ahigh,Bhigh)
    
    if n == 16:
        MEO = RKMul(AEO,BEO)
        
        M0 = SBMul(A0,B0)
        M1 = SBMul(A1,B1)
        M2 = SBMul(A2,B2)
        M3 = SBMul(A3,B3)
        ML = SBMul(Alow,Blow)
        MH = SBMul(Ahigh,Bhigh)
    
    # Step 5
    P1 = [0 for i in range(5*m)]
    #Put your codes here

    
    # Step 6
    P2 = [0 for i in range(6*m)]
    #Put your codes here
    
    
    # Step 7
    P3 = [0 for i in range(6*m)]
    #Put your codes here
    
    # Step 8
    P4 = [0 for i in range(8*m)]
    #Put your codes here

    
    # Step 9
    #Put your codes here

    
    return C

# Test for TLKMul

In [ ]:
A = GenerateGPolynomial(1024)
B = GenerateGPolynomial(1024)
check(SBMul(A,B),TLKMul(A,B))

# Overall Test for Algorithm Correctness

In [9]:
# Need some time, please wait patiently
# You can reduce the iteration times to have a quick test 
for i in range(100):
    A = GenerateGPolynomial(1024)
    B = GenerateGPolynomial(1024)
    stdr = SBMul(A,B)
    check(stdr,KMul(A,B))
    check(stdr,RKMul(A,B))
    check(stdr,OFKMul(A,B))
    #check(stdr,TLKMul(A,B))
print("Congratulations! Your Implementation is Correct!")

check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
check pass!
chec

# Speed Competition

In [11]:
A = GenerateGPolynomial(8192)
B = GenerateGPolynomial(8192)

start= time.time()
SBMul(A,B)
end = time.time()
print("Solving 8192-degree polynomial multiplication with SBMul needs {0} seconds.".format(end-start))

start= time.time()
KMul(A,B)
end = time.time()
print("Solving 8192-degree polynomial multiplication with KMul needs {0} seconds.".format(end-start))

start= time.time()
RKMul(A,B)
end = time.time()
print("Solving 8192-degree polynomial multiplication with RKMul needs {0} seconds.".format(end-start))

start= time.time()
OFKMul(A,B)
end = time.time()
print("Solving 8192-degree polynomial multiplication with OFKMul needs {0} seconds.".format(end-start))

# start= time.time()
# TLKMul(A,B)
# end = time.time()
# print("Solving 8192-degree polynomial multiplication with TLKMul needs {0} seconds.".format(end-start))

Solving 8192-degree polynomial multiplication with SBMul needs 14.793597221374512 seconds.
Solving 8192-degree polynomial multiplication with KMul needs 2.7072460651397705 seconds.
Solving 8192-degree polynomial multiplication with RKMul needs 5.619056940078735 seconds.
Solving 8192-degree polynomial multiplication with OFKMul needs 4.406430244445801 seconds.
